In [0]:
# from google.colab import files
# files.upload()

In [0]:
batch_size = 64  # Batch size for training.
epochs = 30  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path,encoding='utf-8').read().split('\n')

In [0]:
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
     
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
  
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [0]:
len(input_characters)

71

In [0]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [0]:
import numpy as np
import keras
from keras.layers import LSTM

Using TensorFlow backend.


In [0]:
encoder_input_data = np.zeros(
                            (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
                            dtype='float32')

decoder_input_data = np.zeros(
                            (len(target_texts),max_decoder_seq_length,num_decoder_tokens),
                            dtype='float32')

decoder_target_data = np.zeros(
                            (len(target_texts),max_decoder_seq_length,num_decoder_tokens),
                            dtype='float32')

In [0]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]] = 1
  
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1
      

In [0]:
from keras.layers import Input
from keras.layers import Dense

In [0]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_state = [state_h, state_c]

In [0]:
decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs, _ , _ = decoder_lstm(decoder_inputs,initial_state=encoder_state)

decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
from keras.models import Model

In [0]:
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [0]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.8761 - val_loss: 0.9266
Epoch 2/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.7036 - val_loss: 0.7752
Epoch 3/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.6089 - val_loss: 0.7033
Epoch 4/30
1984/8000 [======>.......................] - ETA: 15s - loss: 0.5757

8000/8000 [==============================] - 22s 3ms/step - loss: 0.5569 - val_loss: 0.6535
Epoch 5/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.5180 - val_loss: 0.6199
Epoch 6/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4870 - val_loss: 0.6207
Epoch 7/30
6400/8000 [=======================>......] - ETA: 4s - loss: 0.4633

8000/8000 [==============================] - 22s 3ms/step - loss: 0.4621 - val_loss: 0.5796
Epoch 8/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4405 - val_loss: 0.5495
Epoch 9/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4209 - val_loss: 0.5344
Epoch 10/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.4066

8000/8000 [==============================] - 22s 3ms/step - loss: 0.4047 - val_loss: 0.5202
Epoch 11/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3892 - val_loss: 0.5103
Epoch 12/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3757 - val_loss: 0.4994
Epoch 13/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.3628

8000/8000 [==============================] - 22s 3ms/step - loss: 0.3630 - val_loss: 0.4942
Epoch 14/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3508 - val_loss: 0.4882
Epoch 15/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3396 - val_loss: 0.4827
Epoch 16/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.3295

8000/8000 [==============================] - 22s 3ms/step - loss: 0.3287 - val_loss: 0.4783
Epoch 17/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3186 - val_loss: 0.4755
Epoch 18/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3085 - val_loss: 0.4732
Epoch 19/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.2989

8000/8000 [==============================] - 22s 3ms/step - loss: 0.2989 - val_loss: 0.4759
Epoch 20/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2900 - val_loss: 0.4726
Epoch 21/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2814 - val_loss: 0.4672
Epoch 22/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.2729

8000/8000 [==============================] - 22s 3ms/step - loss: 0.2730 - val_loss: 0.4690
Epoch 23/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2650 - val_loss: 0.4672
Epoch 24/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2578 - val_loss: 0.4773
Epoch 25/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.2505

8000/8000 [==============================] - 22s 3ms/step - loss: 0.2502 - val_loss: 0.4728
Epoch 26/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2433 - val_loss: 0.4778
Epoch 27/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2365 - val_loss: 0.4715
Epoch 28/30
7232/8000 [==========================>...] - ETA: 1s - loss: 0.2295

8000/8000 [==============================] - 22s 3ms/step - loss: 0.2301 - val_loss: 0.4737
Epoch 29/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2235 - val_loss: 0.4742
Epoch 30/30
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2177 - val_loss: 0.4795
